# DS 5001 Module 9 Lab: FastText

We create word embeddings with novel data using word2vec and visualize results with tSNE.

## Set Up

In [1]:
data_in = './data_in'
data_out = './data_out'
data_prefix = 'novels'

In [2]:
OHCO = ['book', 'chapter', 'para_num', 'sent_num', 'token_num']
PARA = OHCO[:4] # Paragraphs
SENT = OHCO[:5] # Sentences

In [3]:
BAG = PARA

In [4]:
import pandas as pd
import numpy as np
from gensim.models import FastText
from sklearn.manifold import TSNE
import plotly_express as px

## Process

### Import Gensim Corpus

In [5]:
DOCS = pd.read_csv('./data_out/novels-GENSIM_DOCS.csv', header=None)[0].to_list()

In [6]:
VOCAB = pd.read_csv('./data_in/novels-VOCAB.csv')

In [7]:
V = VOCAB[VOCAB.pos_max.str.match(r'^(NN[^P]|VB)')].sort_values('tfidf_max', ascending=False).head(1000)\
    .term_str.to_list()

## Generate word embeddings with Gensim's FastText

In [8]:
FastText?

Init signature:
FastText(
    sentences=None,
    corpus_file=None,
    sg=0,
    hs=0,
    size=100,
    alpha=0.025,
    window=5,
    min_count=5,
    max_vocab_size=None,
    word_ngrams=1,
    sample=0.001,
    seed=1,
    workers=3,
    min_alpha=0.0001,
    negative=5,
    ns_exponent=0.75,
    cbow_mean=1,
    hashfxn=<built-in function hash>,
    iter=5,
    null_word=0,
    min_n=3,
    max_n=6,
    sorted_vocab=1,
    bucket=2000000,
    trim_rule=None,
    batch_words=10000,
    callbacks=(),
    compatible_hash=True,
)
Docstring:     
Train, use and evaluate word representations learned using the method
described in `Enriching Word Vectors with Subword Information <https://arxiv.org/abs/1607.04606>`_, aka FastText.

The model can be stored/loaded via its :meth:`~gensim.models.fasttext.FastText.save` and
:meth:`~gensim.models.fasttext.FastText.load` methods, or loaded from a format compatible with the original
Fasttext implementation via :func:`~gensim.models.fasttext.load_

In [ ]:
model = FastText(sentences=DOCS)

## Visualize with tSNE

### Generate coordinates to plot

In [ ]:
coords = pd.DataFrame(index=range(1000))
coords['label'] = V
coords['vector'] = coords['label'].apply(lambda x: model.wv[x])

In [ ]:
coords.head()

### Use ScikitLearn's TSNE library

In [ ]:
tsne_engine = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
tsne_model = tsne_engine.fit_transform(coords['vector'].tolist())

In [ ]:
coords['x'] = tsne_model[:,0]
coords['y'] = tsne_model[:,1]

In [ ]:
coords.head()

### Plot the coordinates

In [ ]:
px.scatter(coords, 'x', 'y', text='label', height=1000).update_traces(mode='text')

## Semantic Algebra

### Analogies

$A : B :: C : D? \rightarrow B - A + C = D$


In [ ]:
def complete_analogy(A, B, C, n=2):
    try:
        cols = ['term', 'sim']
        return pd.DataFrame(model.wv.most_similar(positive=[B, C], negative=[A])[:n], columns=cols)
    except KeyError as e:
        print('Error:', e)
        return None
    
def get_most_similar(positive, negative=None):
    return pd.DataFrame(model.wv.most_similar(positive, negative), columns=['term', 'sim'])

In [ ]:
complete_analogy('man', 'boy', 'woman', 3)

In [ ]:
complete_analogy('girl', 'daughter', 'boy', 3)

In [ ]:
complete_analogy('girl', 'sister', 'boy', 3)

In [ ]:
complete_analogy('man', 'gentleman', 'woman', 5)

In [ ]:
complete_analogy('woman', 'lady', 'man', 5)

In [ ]:
complete_analogy('day', 'sun', 'night', 5)

### Similarites

In [ ]:
get_most_similar('joy')

In [ ]:
get_most_similar('man')

In [ ]:
get_most_similar(['woman','girl'], ['man'])

In [ ]:
get_most_similar(positive=['man'], negative=['woman'])

In [ ]:
get_most_similar(positive=['woman'], negative=['girl'])

In [ ]:
get_most_similar(positive='woman')

In [ ]:
get_most_similar('woman')

In [ ]:
get_most_similar(['woman'],['marriage'])

In [ ]:
get_most_similar(['woman'],['lady'])

In [ ]:
get_most_similar(['man'],['gentleman'])

## Save

In [ ]:
W2V = pd.DataFrame(model.wv.vectors_norm, index=model.wv.vocab.keys())
W2V.to_csv('{}/{}-W2V.csv'.format(data_out, data_prefix))
pd.Series(DOCS).to_csv('{}/{}-GENSIM_DOCS.csv'.format(data_out, data_prefix), index=False, header=False)